In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import generation.constants as const
from generation.clue_processor import CollectiveClueProcessor

for source in const.CLUE_SOURCES:
    source['path'] = os.path.join(const.DATA_ROOT, source['file_name'])
clue_processor = CollectiveClueProcessor(const.CLUE_SOURCES)

all_words = []
for v in clue_processor.words.values():
    all_words.extend(list(v['all']))
all_words

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

letter_freq_raw = pd.read_csv('../data/letter-frequencies.csv')
letter_freq = {e.letter: e.freq for _, e in letter_freq_raw.iterrows()}
inflation_scores = {word: sum(letter_freq[c] for c in word)/len(word) for word in all_words}
plt.hist(inflation_scores.values(), bins=50)
plt.show()

In [ ]:
with open('results/inflated-words.txt', 'w') as f:
    inflated_words = {k: v for k, v in inflation_scores.items() if v > 0.075}
    f.write(str(len(inflated_words)))
    f.write(str(inflated_words))

In [ ]:
import wordninja
def split_words(s):
    contractions = ['d', 'm', 's', 't', 'll', 're', 've']
    split = wordninja.split(s)
    for i in range(len(split)-1, 0, -1):
        if split[i] in contractions:
            split[i-1] += split.pop(i)
    return ' '.join(split)

split_words('zees')

In [ ]:
from generation.clue_processor import CollectiveClueProcessor
import generation.constants as const

for source in const.CLUE_SOURCES:
    source['path'] = os.path.join(const.DATA_ROOT, source['file_name'])

clue_processor = CollectiveClueProcessor(const.CLUE_SOURCES)
clue_processor.clues

In [ ]:
answers = pd.DataFrame(clue_processor.clues['answer'].unique()).rename(columns={0: 'answer'})
answers['answer_sep'] = answers['answer'].apply(split_words)
answers

In [ ]:
from pytrends.request import TrendReq

requests_args = {
    'headers': {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    }
}
pytrends = TrendReq(requests_args=requests_args)

In [ ]:
progress = None

In [ ]:
import numpy as np
from tqdm import tqdm

def get_popularity(df, chunk_length=10, verbose=False):
    def popularity(terms: set[str], timeframe='today 5-y'):
        pytrends.build_payload(kw_list=terms)
        try:
            return int(pytrends.interest_over_time()[list(terms)].sum().sum())
        except KeyError:
            return 0

    global progress
    chunks = np.array_split(df, len(df)//chunk_length)
    for chunk in tqdm(chunks):
        chunk['popularity'] = chunk[['answer', 'answer_sep']].values.tolist()
        chunk['popularity'] = chunk['popularity'].apply(lambda terms: popularity(set(terms)))
        if verbose:
            print(chunk)
        if progress is None:
            progress = chunk
        else:
            progress = pd.concat([progress, chunk])
    
    return progress

In [ ]:
get_popularity(answers)